In [1]:
import foursquare as fs
import pandas as pd
import time

In [2]:
client_id = 'your_client_id'
client_secret = 'your_client_secret'
redirect_id = 'site_of_your_app'

client = fs.Foursquare(client_id=client_id, client_secret=client_secret, 
                       redirect_uri=redirect_id)

In [3]:
# returns a pandas DF with selected user's personal information
# given list of user's IDs
# free API limit is 500 (every query resets in an hour).
def user_info(client, user_id_list):
    
    df = pd.DataFrame(columns=['UserId', 'HomeCity', 'Gender', 'FriendCount',
                               'CheckInCount', 'ListCount', 'PhotoCount', 
                               'TipCount'])
    # count is for deleted users (removed their accont but left a review)
    count = 0
    # count_user is for printing the progress (see try-clause)
    count_user = 0
    for user in user_id_list:
        
        try:
            person = client.users(user)['user']
            
            count_user += 1
            print(count_user, end=',')
            
            HomeCity = person['homeCity']
            Gender = person['gender']
            Friends = person['friends']['count']
            Chekins = person['checkins']['count']
            Lists = person['lists']['count']
            Photos = person['photos']['count']
            Tips = person['tips']['count']
            
            df = df.append([{'UserId' : user, 'HomeCity' : HomeCity, 
                             'Gender' : Gender, 'FriendCount' : Friends,
                             'CheckInCount' : Chekins, 'ListCount' : Lists, 
                             'PhotoCount' : Photos, 'TipCount': Tips}])
        
        # runs if the acc of a user has been deleted
        except fs.ParamError:
            count += 1
            print('ParamError. Deleted', user)
            continue
        
        # runs if the API limit prevents new queries (sleep one hour)
        except fs.RateLimitExceeded:
            print('waiting for an hour...')
            time.sleep(60 * 60)
            print('continue')
            
            person = client.users(user)['user']
            
            HomeCity = person['homeCity']
            Gender = person['gender']
            Friends = person['friends']['count']
            Chekins = person['checkins']['count']
            Lists = person['lists']['count']
            Photos = person['photos']['count']
            Tips = person['tips']['count']
            
            df = df.append([{'UserId' : user, 'HomeCity' : HomeCity, 
                             'Gender' : Gender, 'FriendCount' : Friends,
                             'CheckInCount' : Chekins, 'ListCount' : Lists, 
                             'PhotoCount' : Photos, 'TipCount': Tips}])
        
    print('Deleted:', count, 'of', len(user_id_list))
    return df

In [4]:
# let's select some users just for example purpose
visitors_lst = ['75385428', '25447207', '160789350', '89006174',
                '62162935', '75308640', '125015135', '46274854',
                '34676525', '1906600', '64991911', '15329174', 
                '8357163', '33879706', '54043864']

In [5]:
# downloading information for given users
# users_meta = pd.read_csv('users_meta.csv')
users_meta = user_info(client, user_id_list=visitors_lst)

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,Deleted: 0 of 15


In [6]:
# example of what one may gets
users_meta.head()

,UserId,HomeCity,Gender,FriendCount,CheckInCount,ListCount,MajorCount,PhotoCount,TipCount
0,75385428,"Saint-Petersburg, St.-Petersburg",female,47.0,4611.0,6.0,0.0,0.0,362.0
0,25447207,City of St. Petersburg,female,9.0,137.0,2.0,0.0,0.0,38.0
0,160789350,Genève / Sion,male,0.0,331.0,2.0,0.0,0.0,201.0
0,89006174,"Saint Petersburg, Russia",female,30.0,1226.0,5.0,0.0,0.0,201.0
0,62162935,Saint-Petersburg,male,184.0,1081.0,8.0,0.0,0.0,186.0
